In [1]:
import pandas as pd
from glasspy.predict import GlassNet
from glasspy.data import SciGlass
from functools import partial

from funcoes import populacao_cnb as cria_populacao
from funcoes import funcao_objetivo_pop_cnb as funcao_objetivo_pop
from funcoes import selecao_torneio_max as funcao_selecao
from funcoes import cruzamento_ponto_duplo as funcao_cruzamento
from funcoes import mutacao_sucessiva_cnb as funcao_mutacao

In [2]:
NUM_COMPOUNDS = 80
VALOR_MAX_COMPOUNDS = 100

TAMANHO_POPULACAO = 300
NUM_GERACOES = 5
CHANCE_DE_CRUZAMENTO = 0.5
CHANCE_DE_MUTACAO = 0.05
CHANCE_DE_MUTACAO_POR_GENE = 0.25
TAMANHO_TORNEIO = 3

In [3]:
source = SciGlass()
df = source.data
compounds_list  = list(df['compounds'].columns)

model = GlassNet()

funcao_objetivo = partial(funcao_objetivo_pop, compostos=compounds_list[:80], modelo=model)

In [4]:
populacao = cria_populacao(TAMANHO_POPULACAO, NUM_COMPOUNDS, VALOR_MAX_COMPOUNDS)

hall_da_fama = []

for n in range(NUM_GERACOES):
    print(f'Geração {n}', end='\r')
    
    # Seleção
    fitness = funcao_objetivo(populacao)        
    selecionados = funcao_selecao(populacao, fitness, TAMANHO_TORNEIO)
    
    # Cruzamento
    proxima_geracao = []
    for pai, mae in zip(selecionados[::2], selecionados[1::2]):
        individuo1, individuo2 = funcao_cruzamento(pai, mae, CHANCE_DE_CRUZAMENTO)
        proxima_geracao.append(individuo1)
        proxima_geracao.append(individuo2)
    
    # Mutação
    funcao_mutacao(proxima_geracao, CHANCE_DE_MUTACAO, CHANCE_DE_MUTACAO_POR_GENE, VALOR_MAX_COMPOUNDS)
    
    # Atualização do hall da fama
    fitness = funcao_objetivo(proxima_geracao)
        
    maior_fitness = max(fitness)
    indice = fitness.index(maior_fitness)
    hall_da_fama.append(proxima_geracao[indice])    
    
    # Encerramento
    populacao = proxima_geracao
    
    
fitness = funcao_objetivo(hall_da_fama)
maior_fitness = max(fitness)
indice = fitness.index(maior_fitness)
melhor_individuo_observado = hall_da_fama[indice]

melhor_individuo_observado

[94,
 61,
 50,
 16,
 1,
 86,
 89,
 68,
 48,
 33,
 5,
 0,
 45,
 71,
 62,
 32,
 20,
 85,
 100,
 7,
 75,
 92,
 24,
 95,
 90,
 42,
 70,
 67,
 1,
 43,
 73,
 13,
 35,
 42,
 72,
 96,
 99,
 91,
 38,
 3,
 89,
 1,
 74,
 7,
 23,
 67,
 37,
 62,
 45,
 4,
 21,
 56,
 58,
 24,
 82,
 19,
 31,
 26,
 92,
 65,
 56,
 81,
 40,
 17,
 38,
 50,
 0,
 19,
 17,
 71,
 57,
 80,
 23,
 3,
 0,
 60,
 83,
 43,
 24,
 66]

In [5]:
dict_composition = dict(zip(compounds_list[:80], melhor_individuo_observado))
display(list(dict_composition.items()))
predictions = model.predict(dict_composition)
predictions

[('SiO2', 94),
 ('P2O5', 61),
 ('ZrO2', 50),
 ('Na2O', 16),
 ('Al2O3', 1),
 ('Fe2O3', 86),
 ('CaO', 89),
 ('MgO', 68),
 ('K2O', 48),
 ('MnO', 33),
 ('GeO2', 5),
 ('Li2O', 0),
 ('Ta2O5', 45),
 ('ZnO', 71),
 ('SrO', 62),
 ('CdO', 32),
 ('SnO2', 20),
 ('B2O3', 85),
 ('La2O3', 100),
 ('Ga2O3', 7),
 ('Y2O3', 75),
 ('TiO2', 92),
 ('Nb2O5', 24),
 ('PbO', 95),
 ('HfO2', 90),
 ('WO3', 42),
 ('Sb2O3', 70),
 ('Bi2O3', 67),
 ('CdCl2', 1),
 ('KCl', 43),
 ('BaCl2', 73),
 ('CsCl', 13),
 ('TlCl', 35),
 ('CaCl2', 42),
 ('CdI2', 72),
 ('CsI', 96),
 ('BaI2', 99),
 ('PbI2', 91),
 ('CdBr2', 38),
 ('KBr', 3),
 ('CsBr', 89),
 ('BaBr2', 1),
 ('CdF2', 74),
 ('NaCl', 7),
 ('BaF2', 23),
 ('PbF2', 67),
 ('PbCl2', 37),
 ('NaF', 62),
 ('InF3', 45),
 ('GaF3', 4),
 ('ZnF2', 21),
 ('LaF3', 56),
 ('YF3', 58),
 ('BaO', 24),
 ('Cr2O3', 82),
 ('AlF3', 19),
 ('SiF4', 31),
 ('Cu2O', 26),
 ('CaF2', 92),
 ('BeO', 65),
 ('CuO', 56),
 ('Nd2O3', 81),
 ('CeO2', 40),
 ('Cs2O', 17),
 ('As2O3', 38),
 ('SO3', 50),
 ('F', 0),
 ('NH4NO

,T0,T1,T2,T3,T4,T5,T6,T7,T8,T9,...,Cp1673K,TMaxGrowthVelocity,MaxGrowthVelocity,CrystallizationPeak,CrystallizationOnset,SurfaceTensionAboveTg,SurfaceTension1173K,SurfaceTension1473K,SurfaceTension1573K,SurfaceTension1673K
0,1835.50324,1479.25487,1263.374911,1154.076869,1135.674629,1069.369922,953.194741,910.173479,880.139097,876.284102,...,1247.19878,1036.606634,-6.230045,846.033189,1207.560933,0.384958,0.316878,0.283462,0.309616,0.966066


In [6]:
float(predictions['Density293K'].iloc[0])

3.9491198745444542